In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers==3.0.0

In [ ]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertPreTrainedModel

In [ ]:
import torch
from torch import nn

In [ ]:
'''
class FinanceClassification(BertPreTrainedModel):
  def __init__(self, config, num_labels = 2):
    super(FinanceClassification, self).__init__(config)
    self.num_labels = num_labels
    self.bert = BertModel(config)
    self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
    self.classifier = torch.nn.Linear(config.hidden_size, num_labels )
    self.fc = nn.Linear(768, 8)
    self.soft = nn.Softmax()

  def forward(self, input_ids, token_type_ids = None, attention_mask = None, labels=None):
    output = self.bert(input_ids, token_type_ids, attention_mask)
    output = output.last_hidden_state

    cls_output = self.soft(output)
    cls_output = self.dropout(cls_output[:,0])
    logits = self.classifier(cls_output)

    if labels is not None:
      loss_fct = BCEWithLogitLoss()
      loss = loss_fct(logits.view(-1, self.num_labels),labels.view(-1, self.num_labels))
      return loss
    else:
      return logits
'''    
    



'\nclass FinanceClassification(BertPreTrainedModel):\n  def __init__(self, config, num_labels = 2):\n    super(FinanceClassification, self).__init__(config)\n    self.num_labels = num_labels\n    self.bert = BertModel(config)\n    self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)\n    self.classifier = torch.nn.Linear(config.hidden_size, num_labels )\n    self.fc = nn.Linear(768, 8)\n    self.soft = nn.Softmax()\n\n  def forward(self, input_ids, token_type_ids = None, attention_mask = None, labels=None):\n    output = self.bert(input_ids, token_type_ids, attention_mask)\n    output = output.last_hidden_state\n\n    cls_output = self.soft(output)\n    cls_output = self.dropout(cls_output[:,0])\n    logits = self.classifier(cls_output)\n\n    if labels is not None:\n      loss_fct = BCEWithLogitLoss()\n      loss = loss_fct(logits.view(-1, self.num_labels),labels.view(-1, self.num_labels))\n      return loss\n    else:\n      return logits\n'

In [ ]:
nlabel = 2

In [ ]:
'''
from transformers import BertConfig
config = BertConfig.from_pretrained("bert-base-uncased")
config.is_decoder = True
model = FinanceClassification(config, num_labels=nlabel)
'''

'\nfrom transformers import BertConfig\nconfig = BertConfig.from_pretrained("bert-base-uncased")\nconfig.is_decoder = True\nmodel = FinanceClassification(config, num_labels=nlabel)\n'

In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from torch import Tensor

In [ ]:
'''
a=[{'a':1, 'b':2}, {'a':3, 'b':4}]
df = DataFrame(a, columns = ['a','b'])
print(df)
df = df.set_index('b')
print(df)
'''

"\na=[{'a':1, 'b':2}, {'a':3, 'b':4}]\ndf = DataFrame(a, columns = ['a','b'])\nprint(df)\ndf = df.set_index('b')\nprint(df)\n"

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('there are %d GPU(s) available.'%torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print('No GPU available')

there are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
train = pd.read_csv("./drive/My Drive/Colab Notebooks/glue_data/Combined_News_DJIA.csv")
train = train.dropna(axis = 0)

In [ ]:
train = train.loc[:,['Label','Top1']]
print(train.columns)

Index(['Label', 'Top1'], dtype='object')


In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(train, test_size = 0.3, stratify=train['Label'])

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
docu_bert = ["[CLS]"+str(s)+"[SEP]" for s in train_set.Top1]
docu_bert[:5]

["[CLS]Update: U.S. Customs Won't Apologize for Destroying Musician's Rare Flutes[SEP]",
 '[CLS]North Korea World Cup team shamed in six-hour grilling in front of 400 government officials, students and journalists. Coach punished for "betraying" Kim Jong-un. But they got off lightly apparently...[SEP]',
 '[CLS]A 28-year-old Tunisian who was caught on security camera stealing from a man who had fallen onto the tracks on the Stockholm subway and then leaving him to be hit by a train was sentenced to prison, deportation[SEP]',
 '[CLS]NASA-backed fusion engine could cut Mars trip down to 30 days[SEP]',
 '[CLS]For 23rd time, U.N. nations urge end to U.S. embargo on Cuba[SEP]']

In [ ]:
device = torch.device("cuda")
tokenizer= BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased')
tokenized_texts = [tokenizer.tokenize(s) for s in docu_bert]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
print(tokenized_texts)

[['[CLS]', 'Up', '##date', ':', 'U', '.', 'S', '.', 'Customs', 'Won', "'", 't', 'A', '##pol', '##og', '##ize', 'for', 'Des', '##tro', '##ying', 'Music', '##ian', "'", 's', 'Rare', 'Flute', '##s', '[SEP]'], ['[CLS]', 'North', 'Korea', 'World', 'Cup', 'team', 'shame', '##d', 'in', 'six', '-', 'hour', 'g', '##rill', '##ing', 'in', 'front', 'of', '400', 'government', 'officials', ',', 'students', 'and', 'journalists', '.', 'Coach', 'punished', 'for', '"', 'betray', '##ing', '"', 'Kim', 'Jong', '-', 'un', '.', 'But', 'they', 'got', 'off', 'lightly', 'apparently', '.', '.', '.', '[SEP]'], ['[CLS]', 'A', '28', '-', 'year', '-', 'old', 'Tunisian', 'who', 'was', 'caught', 'on', 'security', 'camera', 'stealing', 'from', 'a', 'man', 'who', 'had', 'fallen', 'onto', 'the', 'tracks', 'on', 'the', 'Stockholm', 'subway', 'and', 'then', 'leaving', 'him', 'to', 'be', 'hit', 'by', 'a', 'train', 'was', 'sentenced', 'to', 'prison', ',', 'deportation', '[SEP]'], ['[CLS]', 'NASA', '-', 'backed', 'fusion', 'e

In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
MAX_LEN = 128
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long',truncating='post', padding='post')

In [ ]:
input_ids[0]

array([  101,  3725,  9216,   131,   158,   119,   156,   119, 17050,
        9083,   112,   189,   138, 23043,  8032,  3708,  1111, 14177,
        8005,  9257,  1953,  1811,   112,   188, 25574, 25058,  1116,
         102,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [ ]:
from torch.optim import Adam
import torch.nn.functional as F



In [ ]:
attention_masks = []
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, train_set['Label'].values, random_state=42, test_size=0.1)

In [ ]:
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.1)

In [ ]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
BATCH_SIZE = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

In [ ]:
sentences = test_set['Top1']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = test_set['Label'].values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup
import random
import time
import datetime

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.70
  Training epcoh took: 0:00:15

Running Validation...
  Accuracy: 0.52
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.69
  Training epcoh took: 0:00:15

Running Validation...
  Accuracy: 0.53
  Validation took: 0:00:01

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.67
  Training epcoh took: 0:00:15

Running Validation...
  Accuracy: 0.45
  Validation took: 0:00:01

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.63
  Training epcoh took: 0:00:15

Running Validation...
  Accuracy: 0.48
  Validation took: 0:00:01

Training complete!


In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.49
Test took: 0:00:02


In [ ]:
sets = pd.read_csv("./drive/My Drive/Colab Notebooks/glue_data/data.txt", sep='|')

In [ ]:
print(sets)

            date                                            content  UD
0     2016-03-01  Why Target is going big on this small water-bo...   1
1     2016-03-02  Bank of England digital cash? Watch out for th...   1
2     2016-03-02  California's most iconic burger chain is on th...   1
3     2016-03-02  Here's how you can invest in the blockchain - ...   1
4     2016-03-03  Adidas sees sneaker success, but golf woes - Y...   1
...          ...                                                ...  ..
5181  2021-02-25  Target will open mini Apple stores to attract ...   0
5182  2021-02-25  Is Now the Time To Buy These Popular Stocks? -...   0
5183  2021-02-25  2021 Prophet Brand Relevance Index® Features M...   0
5184  2021-02-25  The CACP Endorses the Child Search Network to ...   0
5185  2021-02-25  Microsoft facing issues with Teams livestreami...   0

[5186 rows x 3 columns]


In [ ]:
dates = sets['date'].unique()

In [ ]:
df = DataFrame(dates)

In [ ]:
pred_doc = ["[CLS]"+str(s)+"[SEP]" for s in sets.content]

In [ ]:

tokenized_texts = [tokenizer.tokenize(s) for s in pred_doc]
print(tokenized_texts[0])

['[CLS]', 'Why', 'Target', 'is', 'going', 'big', 'on', 'this', 'small', 'water', '-', 'bottle', 'start', '##up', '-', 'Yahoo', 'Finance', '[SEP]']


In [ ]:
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

array([  101,  2009, 18525,  1110,  1280,  1992,  1113,  1142,  1353,
        1447,   118,  5346,  1838,  4455,   118, 23535,  7476,   102,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [ ]:
outs = []
model.eval()
for i in range(5186):
  print(i)
  with torch.no_grad():
    out = model(torch.tensor(input_ids[i:i+1]).to(device))
  outs.append(out[0].detach().cpu().numpy())

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426


In [ ]:
print(outs)

[array([[ 0.0770118 , -0.12544265]], dtype=float32), array([[0.08132041, 0.08672579]], dtype=float32), array([[0.09327006, 0.12103747]], dtype=float32), array([[ 0.01826615, -0.0737208 ]], dtype=float32), array([[ 0.04865152, -0.08359052]], dtype=float32), array([[ 0.07183134, -0.11342634]], dtype=float32), array([[ 0.07032574, -0.01232918]], dtype=float32), array([[0.05075454, 0.07481597]], dtype=float32), array([[ 0.10610314, -0.10213161]], dtype=float32), array([[ 0.05814855, -0.1176768 ]], dtype=float32), array([[0.11415654, 0.06107249]], dtype=float32), array([[0.05318685, 0.00045551]], dtype=float32), array([[0.02924483, 0.2789735 ]], dtype=float32), array([[0.05081458, 0.01213244]], dtype=float32), array([[0.08632805, 0.03780164]], dtype=float32), array([[ 0.03395578, -0.08930122]], dtype=float32), array([[ 0.04098571, -0.09026857]], dtype=float32), array([[ 0.05140325, -0.11336269]], dtype=float32), array([[0.0863545 , 0.05072546]], dtype=float32), array([[0.09587587, 0.1215592

In [ ]:
labels = []
for logits in outs:
  labels.append(np.argmax(logits, axis = 1).flatten().item())

In [ ]:
print(labels)

[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 

In [ ]:
sets['label'] = labels

In [ ]:
print(sets)

            date                                            content  UD  label
0     2016-03-01  Why Target is going big on this small water-bo...   1      0
1     2016-03-02  Bank of England digital cash? Watch out for th...   1      1
2     2016-03-02  California's most iconic burger chain is on th...   1      1
3     2016-03-02  Here's how you can invest in the blockchain - ...   1      0
4     2016-03-03  Adidas sees sneaker success, but golf woes - Y...   1      0
...          ...                                                ...  ..    ...
5181  2021-02-25  Target will open mini Apple stores to attract ...   0      0
5182  2021-02-25  Is Now the Time To Buy These Popular Stocks? -...   0      0
5183  2021-02-25  2021 Prophet Brand Relevance Index® Features M...   0      1
5184  2021-02-25  The CACP Endorses the Child Search Network to ...   0      1
5185  2021-02-25  Microsoft facing issues with Teams livestreami...   0      0

[5186 rows x 4 columns]


In [ ]:
sets.to_csv("./drive/My Drive/Colab Notebooks/glue_data/labeldata.csv")

In [ ]:
sets = sets.drop_duplicates(["date"])

In [ ]:
sets.to_csv("./drive/My Drive/Colab Notebooks/glue_data/labeldata.csv")

In [ ]:
print(sets)

            date                                            content  UD  label
0     2016-03-01  Why Target is going big on this small water-bo...   1      0
1     2016-03-02  Bank of England digital cash? Watch out for th...   1      1
4     2016-03-03  Adidas sees sneaker success, but golf woes - Y...   1      0
6     2016-03-04  If you're not using these shopping apps from T...   0      0
9     2016-03-07  The stock market for sneakers is here - Yahoo ...   0      0
...          ...                                                ...  ..    ...
5138  2021-02-19  Apple In Talks To Buy Lidar Technology For Sel...   0      1
5139  2021-02-22  CORRECTED-Alphabet in talks with Spanish publi...   0      0
5143  2021-02-23  Apple’s biggest mistake was not buying Netflix...   1      0
5157  2021-02-24  Australia passes law to make Google, Facebook ...   0      1
5172  2021-02-25  Amazon Music lands on Google TV and Android TV...   0      0

[1196 rows x 4 columns]
